In [1]:
%run 'connect.ipynb'

In [2]:
import datetime
starttime = datetime.datetime.now()
from tqdm.notebook import tnrange
for i in tnrange(100):
    trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

elapsed = datetime.datetime.now() - starttime
print('Elapsed time: %1.3f seconds; %5.1f captures/second' % (elapsed.total_seconds(), scope.adc.segments/elapsed.total_seconds()))

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time: 3.278 seconds;   0.3 captures/second


In [3]:
cw.program_target(scope, cw.programmers.SAM4SProgrammer, 'fw/simpleserial-aes-SSv2.hex')
target.dis()
target = cw.target(scope, cw.targets.SimpleSerial2)
reset_target(scope)

In [4]:
starttime = datetime.datetime.now()
from tqdm.notebook import tnrange
for i in tnrange(100):
    trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

elapsed = datetime.datetime.now() - starttime
print('Elapsed time: %1.3f seconds; %5.1f captures/second' % (elapsed.total_seconds(), scope.adc.segments/elapsed.total_seconds()))

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time: 1.981 seconds;   0.5 captures/second


In [5]:
print('***WARNING: this is a bad example, DO NOT BORROW IT!***')

scope.adc.segments = 100
scope.adc.samples = 500

scope.arm()

starttime = datetime.datetime.now()
for i in tnrange(scope.adc.segments):
    # make the target encrypt:
    target.simpleserial_write('p', bytearray(16))
    # wait for TIO4 to go high:
    while not scope.io.tio_states[3]:
        pass
    # wait for TIO4 to go back low:
    while scope.io.tio_states[3]:
        pass

elapsed = datetime.datetime.now() - starttime
print('Elapsed time: %1.3f seconds; %5.1f captures/second' % (elapsed.total_seconds(), scope.adc.segments/elapsed.total_seconds()))

# clear things up just in case:
scope.sc.arm(False)
scope.errors.clear()

***WARNING: this is a bad example, DO NOT BORROW IT!***


  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time: 0.449 seconds; 222.7 captures/second


In [6]:
cw.program_target(scope, cw.programmers.SAM4SProgrammer, 'fw/simpleserial-trace-SSv1.hex')
target.dis()
target = cw.target(scope, cw.targets.SimpleSerial)
reset_target(scope)

In [7]:
scope.adc.segments = 100

# tell the target how many encryptions to do (scope.adc.segments):
target.simpleserial_write('n', list(int.to_bytes(scope.adc.segments, length=2, byteorder='big')))
target.simpleserial_wait_ack()

scope.arm()
starttime = datetime.datetime.now()
target.simpleserial_write('f', bytearray(16))
#target.flush()
ret = scope.capture(poll_done=True)
wave = scope.get_last_trace()
resp = target.simpleserial_read('r', 16)
elapsed = datetime.datetime.now() - starttime
print('Elapsed time: %1.3f seconds; %5.1f captures/second' % (elapsed.total_seconds(), scope.adc.segments/elapsed.total_seconds()))

Elapsed time: 0.133 seconds; 753.3 captures/second


In [8]:
scope.adc.samples = 50
scope.adc.segments = 1000

target.simpleserial_write('k', bytearray(16))
target.simpleserial_wait_ack()
target.simpleserial_write('n', list(int.to_bytes(scope.adc.segments, length=2, byteorder='big')))
target.simpleserial_wait_ack()

scope.arm()
starttime = datetime.datetime.now()
target.simpleserial_write('f', bytearray(16))
#target.flush()
ret = scope.capture(poll_done=True)
wave = scope.get_last_trace()
resp = target.simpleserial_read('r', 16)
elapsed = datetime.datetime.now() - starttime
print('Elapsed time: %1.3f seconds; %5.1f captures/second' % (elapsed.total_seconds(), scope.adc.segments/elapsed.total_seconds()))

Elapsed time: 1.125 seconds; 888.6 captures/second


In [9]:
from Crypto.Cipher import AES
cipher = AES.new(bytearray(16), AES.MODE_ECB)
ct = bytearray(16)
for i in range(scope.adc.segments):
    ct = cipher.encrypt(ct)
assert ct == resp

In [10]:
assert not scope.adc.errors, scope.adc.errors